# Assignment 4
### Last updated: April 25, 2022

### Name: Kevin Borah

### Uniqname: kborah

## Instructions

Please turn in:
1. A Jupyter Notebook file. This file should show all of the required work, including code, results, visualizations (if any), and necessary comments to your code. Irrelevant code and results should be deleted prior to submission.

2. An html file showing the preview of the Notebook. To create this file, select File -> Download as > HTML. 

### Before submitting, please select Kernel -> Restart & Run All.

In [1]:
import re
import time
import pickle

from networkx.drawing.nx_pydot import graphviz_layout
from networkx.algorithms import community
from networkx.algorithms.community import modularity
import networkx as nx

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

## <font color='red'>Important</font>

<font color='red' size=2>Please **AVOID** using `community` and `modularity` as your variable names. These are imported as preserved names for networkx submodules. Changing their representations would result in autograder failuers.</font>

In [2]:
# disable warnings
import warnings
warnings.filterwarnings('ignore')

## Part 1. Wikipedia Network with Communities

## Data description

In this assignment, we are going to analyze the community structure of a network. We will use a Wikipedia based [Map of Science](https://figshare.com/articles/A_Wikipedia_Based_Map_of_Science/11638932) network for our exploration. In this network, each node represents a Wikipedia page in a domain of science, such as natural science or social science. An edge exists between two nodes if the cosine similarity of their page contents reaches a pre-defined threshold.

In [3]:
G = nx.read_gml('assets/MapOfScience.gml', label='id')

Each node in the graph contains the attributes:
- "name:" the title of the article 
- "Class" the science domain 
- "WikipediaUrl:" the Wikipedia URL 

Let's look at some examples: 

In [4]:
list(G.nodes(data=True))[0:5]

[(0,
  {'label': '0',
   'name': 'Accounting',
   'Class': 'Applied',
   'WikipediaUrl': 'https://en.wikipedia.org/wiki/Accounting'}),
 (1,
  {'label': '1',
   'name': 'Aerospace engineering',
   'Class': 'Applied',
   'WikipediaUrl': 'https://en.wikipedia.org/wiki/Aerospace_engineering'}),
 (2,
  {'label': '2',
   'name': 'Agricultural engineering',
   'Class': 'Applied',
   'WikipediaUrl': 'https://en.wikipedia.org/wiki/Agricultural_engineering'}),
 (3,
  {'label': '3',
   'name': 'Agricultural science',
   'Class': 'Applied',
   'WikipediaUrl': 'https://en.wikipedia.org/wiki/Agricultural_science'}),
 (4,
  {'label': '4',
   'name': 'Agronomy',
   'Class': 'Applied',
   'WikipediaUrl': 'https://en.wikipedia.org/wiki/Agronomy'})]

In [5]:
(G.nodes(data=True))[0]['Class']

'Applied'

The edges contain the cosine similarity of the text of the two articles：

In [6]:
list(G.edges(data=True))[0:5]

[(0, 21, {'CosineSimilarity': 0.369447477753246}),
 (0, 50, {'CosineSimilarity': 0.395432741205435}),
 (0, 70, {'CosineSimilarity': 0.388758063740006}),
 (0, 88, {'CosineSimilarity': 0.371542879166867}),
 (0, 516, {'CosineSimilarity': 0.365688238862527})]

### Q1. (1 point, Autograded) Let's extract the largest connected component from the above graph. In the following questions, we will focus our analysis on this sub-graph. How many nodes are there in this new network?

In [7]:
# Extract the largest connected component of the original dataset
G = G.subgraph(max(nx.connected_components(G), key=len))
N = len(G.nodes)  # stores the number of nodes in this graph

# YOUR CODE HERE
# raise NotImplementedError()

In [8]:
#hidden tests for Question 1 are within this cell

### Q2. (2 points, Autograded) If we think of science domains as communities, how many communities are there in the network and what are their Classes?

In [9]:
# final = []
# for i in G.nodes:
#     tmp = G.nodes(data=True)[i]['Class']
#     final.append(tmp)

list_of_communities = list(set([G.nodes(data=True)[i]['Class'] for i in G.nodes])) # set of unique community labels
N = len(list_of_communities)         # number of communities in total

# YOUR CODE HERE
# raise NotImplementedError()
list_of_communities

['Social', 'Formal', 'Natural', 'Applied']

In [10]:
#hidden tests for Question 2 are within this cell

### Q3. (4 points, Autograded) How many nodes does each community have?

Hint: you may want to use the [`Counter`](https://docs.python.org/2/library/collections.html#counter-objects) object for this question.

In [11]:
from collections import Counter
dict_num_community = Counter()   # a Counter object with the format {community_name: number_of_nodes}
for i in G.nodes:
    for x in list_of_communities:
        if G.nodes(data=True)[i]['Class'] == x:
            dict_num_community.update({x:1})
            
dict_num_community
# YOUR CODE HERE
# raise NotImplementedError()

Counter({'Applied': 92, 'Formal': 193, 'Natural': 168, 'Social': 224})

In [12]:
#hidden tests for Question 3 are within this cell

## Part 2. Measures of Partition Quality

We discussed 5 ways to measure the quality of a partition: modularity, coverage, performance, separability, and density. 

Modularity, coverage, and performance can be measured using `networkx` functions in the `algorithms.community` module. You can check all the functions provided by a module with the built-in `dir()` function:

```python
from networkx.algorithms import community
dir(community)
>>> ...
 'coverage',
 'modularity',
 'performance',
```

Descriptions of the three measures are provided in the source code,

1. `networkx.algorithms.community.modularity(G, communities, weight='weight')`

2. [`networkx.algorithms.community.quality.coverage(G, partition)`](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.quality.coverage.html#networkx.algorithms.community.quality.coverage)

3. [`networkx.algorithms.community.quality.performance(G, partition)`](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.quality.performance.html#networkx.algorithms.community.quality.performance)





For separability and density, you will implement your own functions. 

Since separability and density first apply a measure to each community and then takes the average over all communities, we provide a helper function `avg_measure(G, communities, measure)` that computes the average value for a given measure over all communities in a graph:

In [13]:
def avg_measure(G, communities, measure):
    """
    Calculate the average value of a given measure across communities in a graph.
    
    Args:
        G - nx graph object; a graph to operate upon.
        communities - list; a collection of sets of nodes that each comprise a community.
        measure - function; calculates a measure for a community in a graph.
    
    Returns:
        (unnamed) - float; the calculated average measure.
    """
    sum_ = 0
    for comm in communities:
        sum_ += measure(G, comm)
    return sum_ / len(communities)

### Q4. (10 points, Autograded) Let's begin implementing the function to measure the separability for a single community. That is, measure the ratio of intra-community to inter-community edges. 

If there are 0 inter-community edges, the function should assume that the actual number is 1, making the separability value equal to the number of intra-community edges. 

Hint: `G.edges(community)`, where `community` is a set of nodes, returns the edges that are incident to at least one node in `community`. That is, it returns the edges that have at least one endpoint in `community`. 

In [14]:
def separability_one_community(G, community):
    """
    Calculate the separability of a community by finding the ratio of
    intra-community edges to inter-community edges.
    
    Args:
        G - nx graph object; a graph to operate upon.
        community - set; a collection of nodes that comprise a community.
    
    Returns:
        result - float; the separability in a given community.
    """
    result = None # assign separability to it and return
    
    intra_comm = []
    inter_comm = []
    
    for i,v in list(G.edges(community)):
        if i not in community or v not in community:
            inter_comm.append((i,v))
        else:
            intra_comm.append((i,v))
                
    if len(inter_comm)==0:
        len_inter = 1
    else:
        len_inter = len(inter_comm)
        
    result = len(intra_comm)/len_inter
    
    return result
            
    

# YOUR CODE HERE
# raise NotImplementedError()

In [15]:
#hidden tests for Question 4 are within this cell

Now we can simply use `avg_measure(G, communities, separability_one_community)` to measure the separability of a partition. 

### Q5. (10 points, Autograded) Let's now implement the function to measure the density of a single community. That is, the fraction of intra-community edges out of all possible edges. 

In [16]:
def density_one_community(G, community):
    """
    Calculate the density of a community by finding the fraction of
    intra-community edges out of all possible edges.
    
    Args:
        G - nx graph object; a graph to operate upon.
        community - set; a collection of nodes that comprise a community.
    
    Returns:
        result - float; the density of a given community.
    """
    
    result = None # assign density to it and return    
    
    if len(community) == 1:  # If the community has only one node, just return 1 
        return 1
    
    # YOUR CODE HERE
#     raise NotImplementedError()

    intra_comm = []
    inter_comm = []
    
    for i,v in list(G.edges(community)):
        if i not in community or v not in community:
            inter_comm.append((i,v))
        else:
            intra_comm.append((i,v))
                
    n = len(community)
    all_poss =  (n*(n-1))/2
    
    result = len(intra_comm)/all_poss
    

    return result




In [17]:
#hidden tests for Question 5 are within this cell

Now we can simply use `avg_measure(G, communities, density_one_community)` to measure the density of a partition. 

### Q6. (5 points, Autograded) What is the modularity, coverage, performance, density, and separability of this network using the science domain as a partition?

In [18]:
communities = []

for x in list_of_communities:
    tmp = []
    for i in G.nodes:
        if G.nodes(data=True)[i]['Class'] == x:
            tmp.append(i)
    communities.append(set(tmp))
        
# measures = [modularity,nx.algorithms.community.quality.coverage,nx.algorithms.community.quality.performance,density_one_community,separability_one_community]

comm = communities

mod = modularity(G,comm)   # modularity
cov = nx.algorithms.community.quality.coverage(G,comm)   # coverage
perf = nx.algorithms.community.quality.performance(G,comm)  # performance
den = avg_measure(G,comm,density_one_community) # density
sep = avg_measure(G,comm,separability_one_community) # separability

mod, cov, perf, den, sep
# communities

# YOUR CODE HERE
# raise NotImplementedError()


(0.39848031219396246,
 0.7152063833052018,
 0.7425423684371532,
 0.0693753209666948,
 1.1403277162954397)

In [19]:
#hidden tests for Question 6 are within this cell

## Part 3. Community Detection Algorithms

Now let's apply community detection algorithms to the Wikipedia graph. For this part, we will ignore the science domains since we are trying to find communities purely based on the structure of the network. 

We will begin with the Girvan-Newman algorithm and pick the partition that results in the largest modularity, as described in the lecture. Since computing this partition takes a long time, we have precomputed the communities in the notebook `Girvan-Newman.ipynb` (stored in the resources folder) and exported the partition to a pickle file. The pickle file is stored as `assets/answer/max_mod_community`. Note that you do not need to run the notebook `Girvan-Newman.ipynb` since we have already done it for you. We only provide it to you as a reference.

### Q7. (1 point, Autograded) Load the partition from `assets/answer/max_mod_community`. How many communities are there in this partition?

**Hint**:

The partition is stored as a pickle file, which can be imported as the following example:
```python
with open("my_pickle_file_name", 'rb') as f:
    my_data = pickle.load(f)

```

In [20]:
with open("assets/answer/max_mod_community", 'rb') as f:
    my_data = pickle.load(f)

num_max_mod_communities = len(my_data)  # number of communities in the partition with the largest modularity value

# YOUR CODE HERE
# raise NotImplementedError()



In [21]:
#hidden tests for Question 7 are within this cell

### Q8. (5 points, Autograded) What is the modularity, coverage, performance, density, and separability of this partition?

In [22]:
mod = None   # modularity
cov = None   # coverage
perf = None  # performance
den = None # density
sep = None # separability

comm = my_data

mod = modularity(G,comm)   # modularity
cov = nx.algorithms.community.quality.coverage(G,comm)   # coverage
perf = nx.algorithms.community.quality.performance(G,comm)  # performance
den = avg_measure(G,comm,density_one_community) # density
sep = avg_measure(G,comm,separability_one_community) # separability


# YOUR CODE HERE
# raise NotImplementedError()

In [23]:
#hidden tests for Question 8 are within this cell

### Q9. (12 points, Autograded) Find a partition of the network with the [label propagation algorithm](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.label_propagation.label_propagation_communities.html) and compute the number of communities in the partition and its modularity, coverage, performance, denisty, and separability. This function uses semi-synchronous updating.

In [24]:
lab_prop = community.label_propagation_communities(G)
comm = list(lab_prop)

num_community = len(comm)

# mod = None   # modularity
# cov = None   # coverage
# perf = None  # performance
# den = None # density
# sep = None # separability

mod = modularity(G,comm)   # modularity
cov = nx.algorithms.community.quality.coverage(G,comm)   # coverage
perf = nx.algorithms.community.quality.performance(G,comm)  # performance
den = avg_measure(G,comm,density_one_community) # density
sep = avg_measure(G,comm,separability_one_community) # separability

# comm

# YOUR CODE HERE
# raise NotImplementedError()

In [25]:
#hidden tests for Question 9 are within this cell

In [26]:
mod

0.5841880852733244

The [Clauset-Newman-Moore greedy modularity maximization algorithm](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.modularity_max.greedy_modularity_communities.html#networkx-algorithms-community-modularity-max-greedy-modularity-communities) implements an Agglomerative Hierarchical Clustering procedure to find a partition with high modularity. 

Note: the function `greedy_modularity_communities` returns a list of `Frozensets`, where each `Frozenset` is simply an immutable Python `set` object (i.e. the elements cannot be modified).



### Q10. (12 points, Autograded) Using Clauset-Newman-Moore greedy modularity maximization, find a partition of the network and compute the number of communities in the partition and its modularity, coverage, performance, density, and separability.

In [27]:
gred_partition = nx.algorithms.community.greedy_modularity_communities(G)

comm = gred_partition

num_community = len(comm) # number of communities in the partition

mod = modularity(G,comm)   # modularity
cov = nx.algorithms.community.quality.coverage(G,comm)   # coverage
perf = nx.algorithms.community.quality.performance(G,comm)  # performance
den = avg_measure(G,comm,density_one_community) # density
sep = avg_measure(G,comm,separability_one_community) # separability


# YOUR CODE HERE
# raise NotImplementedError()



In [28]:
#hidden tests for Question 10 are within this cell

### Q11. (6 points, Autograded) Using [asynchronous Fluid Communities algorithm](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.asyn_fluid.asyn_fluidc.html#networkx.algorithms.community.asyn_fluid.asyn_fluidc) with parameters  `max_iter` = 100 and `seed` = 233 (see Tutorial for details), find the parameter `k` between `k`=3 and `k`=40 that maximizes the modularity of the partition. Note that `k` represents the number of communities in the partition. 

In [29]:
best_k = None #optimal value of k
fluid_communities = None #partition returned by the algorithm with k = best_k

outcomes = {}
for k in range(3,41):
    fluid_partition = list(nx.algorithms.community.asyn_fluidc(G, k, max_iter=100, seed=233))
    mod = modularity(G,fluid_partition)
    outcomes[k] = mod
    
best_k = max(outcomes, key=outcomes.get)
fluid_communities = list(nx.algorithms.community.asyn_fluidc(G, best_k, max_iter=100, seed=233))
    


# YOUR CODE HERE
# raise NotImplementedError()

In [30]:
#hidden tests for Question 11 are within this cell

In [31]:
fluid_communities

[{28,
  46,
  74,
  90,
  97,
  106,
  108,
  149,
  164,
  165,
  210,
  218,
  241,
  246,
  248,
  249,
  256,
  259,
  260,
  261,
  263,
  265,
  266,
  277,
  313,
  345,
  361,
  382,
  408,
  451,
  501,
  541,
  592,
  633,
  636,
  669},
 {17,
  18,
  24,
  39,
  53,
  60,
  76,
  83,
  87,
  292,
  293,
  294,
  295,
  298,
  299,
  308,
  314,
  316,
  324,
  331,
  332,
  334,
  339,
  343,
  347,
  349,
  357,
  359,
  360,
  365,
  366,
  367,
  369,
  384,
  387,
  402,
  403,
  412,
  416,
  423,
  432,
  476,
  548,
  610,
  623},
 {9,
  14,
  20,
  36,
  55,
  68,
  123,
  209,
  243,
  258,
  272,
  286,
  289,
  296,
  300,
  309,
  312,
  323,
  328,
  330,
  335,
  338,
  344,
  353,
  374,
  376,
  378,
  379,
  381,
  386,
  392,
  395,
  397,
  404,
  418,
  420,
  429,
  434,
  435,
  437,
  440,
  448,
  450,
  611},
 {2,
  3,
  4,
  7,
  10,
  12,
  13,
  16,
  26,
  41,
  44,
  66,
  84,
  105,
  109,
  122,
  128,
  130,
  157,
  163,
  212,
  233,
  285,

### Q12. (6 points, Autograded) Using [asynchronous Fluid Communities algorithm](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.asyn_fluid.asyn_fluidc.html#networkx.algorithms.community.asyn_fluid.asyn_fluidc) with parameters  `max_iter` = 100, `seed` = 233, and the value of `k` you found in the previous question, compute the number of communities in the partition and its modularity, coverage, performance, density, and separability.

In [32]:
fluid_partition = list(nx.algorithms.community.asyn_fluidc(G, best_k, max_iter=100, seed=233))

num_community = len(fluid_partition) # number of communities in the partition

comm = fluid_partition

mod = modularity(G,comm)   # modularity
cov = nx.algorithms.community.quality.coverage(G,comm)   # coverage
perf = nx.algorithms.community.quality.performance(G,comm)  # performance
den = avg_measure(G,comm,density_one_community) # density
sep = avg_measure(G,comm,separability_one_community) # separability


# YOUR CODE HERE
# raise NotImplementedError()

In [33]:
#hidden tests for Question 12 are within this cell

### Q13. (6 points, Autograded) Create and print a pandas DataFrame where each row represents a community detection algorithm and each column is a quality measure, as described by the following scheme:


|.|Method name|num_community|modularity|coverage|performance|density|separability|
|-|-|-|-|-|-|-|-|
|0|Girvan–Newman|
|1|Greedy modularity maximization|
|2|Fluid Communities|
|3|Label propogation|


In [34]:
algos = {'Girvan-Newman':my_data, 'Greedy modularity maximization':gred_partition,'Fluid Communities':fluid_partition}

df = []
for i, v in algos.items():
    comm = v
    num_community = len(comm) # number of communities in the partition

    mod = modularity(G,comm)   # modularity
    cov = nx.algorithms.community.quality.coverage(G,comm)   # coverage
    perf = nx.algorithms.community.quality.performance(G,comm)  # performance
    den = avg_measure(G,comm,density_one_community) # density
    sep = avg_measure(G,comm,separability_one_community) # separability
    
    df.append([i,num_community, mod,cov,perf,den,sep])
    
lab_prop = community.label_propagation_communities(G)
comm = list(lab_prop)

num_community = len(comm)

mod = modularity(G,comm)   # modularity
cov = nx.algorithms.community.quality.coverage(G,comm)   # coverage
perf = nx.algorithms.community.quality.performance(G,comm)  # performance
den = avg_measure(G,comm,density_one_community) # density
sep = avg_measure(G,comm,separability_one_community) # separability

df.append(['Label propogation',num_community, mod, cov, perf, den, sep])
    
compare = pd.DataFrame(df,columns=['Method name','numb_community','modularity','coverage','performance','density','separability']) # assign your pandas dataframe here
# YOUR CODE HERE
# raise NotImplementedError()




compare

,Method name,numb_community,modularity,coverage,performance,density,separability
0,Girvan-Newman,35,0.580687,0.815252,0.888706,0.654218,1.079485
1,Greedy modularity maximization,14,0.550356,0.805892,0.775095,0.608520,1.480856
2,Fluid Communities,11,0.590856,0.728249,0.915128,0.211423,1.374656
3,Label propogation,28,0.584188,0.808808,0.857888,0.618999,1.285533


In [35]:
#hidden tests for Question 13 are within this cell

### Q14. (10 points, Manually graded) Does it appear that an algorithm consistently performs better than the others across the different quality measures? Explain. 

You do not need to explain your answer based on the details of the algorithms or quality measures. Do so only based on the quality scores on the dataframe in Q13. 

You should not consider the number of communities to answer this question. 



No, it does not appear that one algorithm consistently outperforms all other algorithms across different quality measures.  All algorithms except for Label Propogation perform best on at least one quality measure.

### Q15. (10 points, Manually graded) Comparing the quality of the partition based on the science domain with that of the partitions generated by the various algorithms, from a network structure perspective, does the science domain appear to be a good way to partition the network into communities? Explain.

Recall that you already computed the quality of the partition based on science domain in Q6. Be sure to use those results when answering this question. 

You should not consider the number of communities to determine if a partition is "good." For this question, focus on the quality scores. 

Science Domain Results
(0.3984803121939624 - Mod 
 0.7152063833052018 - Cov
 0.7425423684371532 - Perf
 0.06937532096669478 - Dens
 1.1403277162954397 - Sep)
 
Based on the quality measure scores, partitioning based on the science domain does not appear to be a good way to partition the network into communities.  The quality measures when partitioning on the science domain fall short of the algorithmic partitioning quality measures across all algorithms.   

# End